In [1]:
import os
import json
import pandas as pd

In [2]:
# modificar segun entorno local
os.chdir( "/Volumes/hd_mvf_datasets/henry_data")

# no modificar
folder_data = "1_external"
folder_pipeline = "2_pipeline"
folder_output = "3_output"

In [3]:
# extraemos nombres de cada estado segun nombre de carpeta
state_name = pd.DataFrame(
    pd.DataFrame(
        os.listdir(folder_data)).rename(columns={0:"state"})["state"].str.split("-").str[1])
state_name.head(2)

,state
0,Indiana
1,Iowa


In [4]:
# valor piloto;
# en el loop seria for i in range(0,max(state_name.index))
i = 0

# creamos una variable con el nombre de estado
# para facilitar el loop para todos los estados
state = state_name["state"][i]
folder_state = "review-" + str(state)

In [5]:
# contamos numero de archivos al interior de cada carpeta del estado
# para facilitar el loop
count_file = 0
# Iterate
for path in os.listdir(os.path.join(folder_data,folder_state)):
    # check if current path is a file
    if os.path.isfile(os.path.join(folder_data,folder_state,path)):
        count_file += 1
count_file

15

In [6]:
# extraemos la informacion
data = []
for f in range(1, count_file):
    with open(
        os.path.join(folder_data,folder_state, str(f) + ".json"), "r") as file:
        for line in file:
            data.append(json.loads(line))
gmaps_state = pd.DataFrame(data)
gmaps_state.info()
gmaps_state.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100000 entries, 0 to 2099999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 128.2+ MB


,user_id,name,time,rating,text,pics,resp,gmap_id
0,111423711419019424734,Larry Sandefur,1615400502414,5,Majestic security was the best job I ever had ...,None,"{'time': 1627580537714, 'text': 'Thanks for th...",0x8816c4b2fb8fb6a1:0x80451636e10ca83f
1,113596625824447012651,John Soard,1597280425295,5,I have used this company for a number of years...,None,"{'time': 1606930574545, 'text': 'Thanks John, ...",0x8816c4b2fb8fb6a1:0x80451636e10ca83f
2,100821699956425809518,Michael Johnson,1596678908723,5,"I am an employee with Majestic Security, and h...",None,"{'time': 1606930806337, 'text': 'Thanks Mike f...",0x8816c4b2fb8fb6a1:0x80451636e10ca83f
3,106063043906772778894,Lauren Moretz,1606588867988,1,My husband has worked for this company for a f...,None,"{'time': 1606930153562, 'text': 'in 33 years, ...",0x8816c4b2fb8fb6a1:0x80451636e10ca83f
4,107227282238314902977,Tracy Workman,1606677082659,5,I'm 55 and have been here with Magestic over 6...,None,"{'time': 1606930223376, 'text': 'Thank You Tra...",0x8816c4b2fb8fb6a1:0x80451636e10ca83f


In [7]:
# indentificamos las columnas de interes y aquellas que se deben desanidar
vars_interes=[
    "gmap_id" # pk google maps
    ,"user_id"
    ,"name"
    ,"time"
    ,"rating"
    ,"text"
    #,"pics" # irrelevante
    #,"resp" # desanidar
    ]

vars_desanidar=[
    "gmap_id" # pk google maps
    ,"resp" # desanidar
    ]


In [8]:
# nos concentramos en: vars_desanidar
gmaps_state_dsndr = pd.DataFrame(gmaps_state[vars_desanidar]).dropna()
gmaps_state_dsndr.info()
gmaps_state_dsndr.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 261715 entries, 0 to 2099965
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   gmap_id  261715 non-null  object
 1   resp     261715 non-null  object
dtypes: object(2)
memory usage: 6.0+ MB


,gmap_id,resp
0,0x8816c4b2fb8fb6a1:0x80451636e10ca83f,"{'time': 1627580537714, 'text': 'Thanks for th..."
1,0x8816c4b2fb8fb6a1:0x80451636e10ca83f,"{'time': 1606930574545, 'text': 'Thanks John, ..."


In [9]:
# se normaliza la columna
gmaps_state_dsndr = gmaps_state_dsndr.set_index("gmap_id")
gmaps_state_dsndda = pd.json_normalize(
    gmaps_state_dsndr["resp"]).set_index(gmaps_state_dsndr.index)
gmaps_state_dsndda.reset_index(inplace=True) # mueve le indice a una columna
# renombramos para facilitar la union mas abajo
gmaps_state_dsndda.rename(
    columns={"time":"resp_time","text":"resp_text"},
    inplace=True
    )
gmaps_state_dsndda.head(2)

,gmap_id,resp_time,resp_text
0,0x8816c4b2fb8fb6a1:0x80451636e10ca83f,1627580537714,"Thanks for the review Larry, and for still bei..."
1,0x8816c4b2fb8fb6a1:0x80451636e10ca83f,1606930574545,"Thanks John, we appreciate working with you an..."


In [10]:
# solo incluimos las variables de interes
gmaps_state_interes = gmaps_state[vars_interes].copy().dropna()
# renombramos para facilitar la union mas abajo
gmaps_state_interes.rename(
    columns={"name":"user_name","time":"user_time","text":"user_text"},
    inplace=True
    )
gmaps_state_interes.info()
gmaps_state_interes.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1158638 entries, 0 to 2099999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   gmap_id    1158638 non-null  object
 1   user_id    1158638 non-null  object
 2   user_name  1158638 non-null  object
 3   user_time  1158638 non-null  int64 
 4   rating     1158638 non-null  int64 
 5   user_text  1158638 non-null  object
dtypes: int64(2), object(4)
memory usage: 61.9+ MB


,gmap_id,user_id,user_name,user_time,rating,user_text
0,0x8816c4b2fb8fb6a1:0x80451636e10ca83f,111423711419019424734,Larry Sandefur,1615400502414,5,Majestic security was the best job I ever had ...
1,0x8816c4b2fb8fb6a1:0x80451636e10ca83f,113596625824447012651,John Soard,1597280425295,5,I have used this company for a number of years...


In [11]:
# unir los dataframes
gmaps_state_norm = pd.merge(
    gmaps_state_interes,
    gmaps_state_dsndda,
    on=["gmap_id"])

# generamos una columna state
# para facilitar la union de todos los datos
gmaps_state_norm["state"] = state

# movemos 'state' a la primera fila para facilitar la visualizacion
first_column = gmaps_state_norm.pop("state")
gmaps_state_norm.insert(0, "state", first_column)

# df final por estado
gmaps_state_norm.to_parquet(os.path.join(
    folder_pipeline, str(folder_state) + "_norm.parquet"))
gmaps_state_norm.info()
print(gmaps_state_norm.isna().sum())
gmaps_state_norm.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39578669 entries, 0 to 39578668
Data columns (total 9 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   state      object
 1   gmap_id    object
 2   user_id    object
 3   user_name  object
 4   user_time  int64 
 5   rating     int64 
 6   user_text  object
 7   resp_time  int64 
 8   resp_text  object
dtypes: int64(3), object(6)
memory usage: 2.7+ GB
state        0
gmap_id      0
user_id      0
user_name    0
user_time    0
rating       0
user_text    0
resp_time    0
resp_text    0
dtype: int64


,state,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text
0,Indiana,0x8816c4b2fb8fb6a1:0x80451636e10ca83f,111423711419019424734,Larry Sandefur,1615400502414,5,Majestic security was the best job I ever had ...,1627580537714,"Thanks for the review Larry, and for still bei..."
1,Indiana,0x8816c4b2fb8fb6a1:0x80451636e10ca83f,111423711419019424734,Larry Sandefur,1615400502414,5,Majestic security was the best job I ever had ...,1606930574545,"Thanks John, we appreciate working with you an..."
